In [14]:
# import necessary library
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, Flatten, ZeroPadding2D, AveragePooling2D, MaxPooling2D, Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import to_categorical
import load_data
import load_testdata

In [2]:
# construc the identity block
def block_iden(X, size_filter, num_filter):
    
    # number of filters used in the 3 conv layer
    f1 = num_filter[0]
    f2 = num_filter[1]
    f3 = num_filter[2]
    
    # store the X as a shortcut first
    X_short = X
    
    X = Conv2D(filters = f1, kernel_size = (1,1), strides = (1,1), padding = 'valid', kernel_initializer = glorot_uniform(seed=None))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters = f2, kernel_size = (size_filter,size_filter), strides = (1,1), padding = 'same', kernel_initializer = glorot_uniform(seed=None))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters = f3, kernel_size = (1,1), strides = (1,1), padding = 'valid', kernel_initializer = glorot_uniform(seed=None))(X)
    X = BatchNormalization(axis = 3)(X)
    
    X = Add()([X, X_short])
    
    X = Activation('relu')(X)
    
    return X

In [3]:
# construct the convolution block
def block_conv(X, size_filter, num_filter, size_stride):
    
    # number of filters used in the 3 conv layer
    f1 = num_filter[0]
    f2 = num_filter[1]
    f3 = num_filter[2]
    
    # store the X as a shortcut first
    X_short = Conv2D(filters = f3, kernel_size = (1,1), strides = (size_stride,size_stride), padding = 'valid', kernel_initializer = glorot_uniform(seed=None))(X)
    X_short = BatchNormalization(axis = 3)(X_short)
    
    
    X = Conv2D(filters = f1, kernel_size = (1,1), strides = (size_stride,size_stride), padding = 'valid', kernel_initializer = glorot_uniform(seed=None))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters = f2, kernel_size = (size_filter,size_filter), strides = (1,1), padding = 'same', kernel_initializer = glorot_uniform(seed=None))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters = f3, kernel_size = (1,1), strides = (1,1), padding = 'valid', kernel_initializer = glorot_uniform(seed=None))(X)
    X = BatchNormalization(axis = 3)(X)
    
    X = Add()([X, X_short])
    
    X = Activation('relu')(X)
    
    return X

In [4]:
# construct the model
def resnet50(shape, classes):
    
    X_input = Input(shape)
    # zero padding
    X = ZeroPadding2D(padding = (3,3))(X_input)
    
    # stage 1
    X = Conv2D(filters = 64, kernel_size = (7,7), strides = (2,2), kernel_initializer = glorot_uniform(seed=None))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size = (3,3), strides = (2,2))(X)
    
    # stage 2
    X = block_conv(X, 3, [64,64,256], 1)
    X = block_iden(X, 3, [64,64,256])
    X = block_iden(X, 3, [64,64,256])
    
    # stage 3
    X = block_conv(X, 3, [128,128,512], 2)
    X = block_iden(X, 3, [128,128,512])
    X = block_iden(X, 3, [128,128,512])
    X = block_iden(X, 3, [128,128,512])
    
    # stage 4
    X = block_conv(X, 3, [256,256,1024], 2)
    X = block_iden(X, 3, [256,256,1024])
    X = block_iden(X, 3, [256,256,1024])
    X = block_iden(X, 3, [256,256,1024])
    X = block_iden(X, 3, [256,256,1024])
    X = block_iden(X, 3, [256,256,1024])
    
    # stage 5
    X = block_conv(X, 3, [512,512,2048], 2)
    X = block_iden(X, 3, [512,512,2048])
    X = block_iden(X, 3, [512,512,2048])
    
    # final stage
    X = AveragePooling2D(pool_size = (2,2))(X)
    X = Flatten()(X)
    X = Dense(classes, activation = 'softmax', kernel_initializer = glorot_uniform(seed=None))(X)
    
    # model
    model = Model(X_input, X, name = 'ResNet50')
    
    return model

In [5]:
model = resnet50((224,224,3), 2)

In [6]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
# model architecture 
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 64) 256         conv2d[0][0]                     
___________________________________________________________________________________________

In [7]:
# load the training data and validation data
x_train, x_test, y_train, y_test = load_data.load_data()

In [8]:
# check the training data shape
x_train.shape

(4185, 224, 224, 3)

In [9]:
# check the tranining label shape
y_train.shape

(4185, 2)

In [11]:
# start the training process
model.fit(x_train, y_train, epochs = 15, batch_size = 128)

Epoch 1/15
33/33 [==============================] - 1309s 40s/step - loss: 1.6320 - accuracy: 0.8048
Epoch 2/15
33/33 [==============================] - 1105s 33s/step - loss: 0.3732 - accuracy: 0.9066
Epoch 3/15
33/33 [==============================] - 982s 30s/step - loss: 0.9202 - accuracy: 0.8442
Epoch 4/15
33/33 [==============================] - 978s 30s/step - loss: 0.4449 - accuracy: 0.9085
Epoch 5/15
33/33 [==============================] - 986s 30s/step - loss: 0.2284 - accuracy: 0.9247
Epoch 6/15
33/33 [==============================] - 985s 30s/step - loss: 0.1559 - accuracy: 0.9415
Epoch 7/15
33/33 [==============================] - 980s 30s/step - loss: 0.1148 - accuracy: 0.9551
Epoch 8/15
33/33 [==============================] - 983s 30s/step - loss: 0.0836 - accuracy: 0.9675
Epoch 9/15
33/33 [==============================] - 997s 30s/step - loss: 0.0573 - accuracy: 0.9771
Epoch 10/15
33/33 [==============================] - 988s 30s/step - loss: 0.0542 - accuracy: 0.97

In [13]:
# evaluate the model using the validation data
preds = model.evaluate(x_test, y_test)
print ("Validation Loss = " + str(preds[0]))
print ("Validation Accuracy = " + str(preds[1]))

33/33 [==============================] - 54s 2s/step - loss: 1.6429 - accuracy: 0.7679
Validation Loss = 1.6428661346435547
Validation Accuracy = 0.7679083347320557
